<a href="https://colab.research.google.com/github/Jiyeon00/2021-1-DL/blob/main/python3_201710180_%EA%B0%95%EC%A7%80%EC%97%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Settings

1) Important required libraries

In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.utils.data as data
import torchvision.datasets as dset
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
import time
import matplotlib.pyplot as plt
import utils

2) Hyperparamter

In [4]:
batch_size = 16 #64 # 1
learning_rate = 0.0001
epoch = 20

n_node = 1024
dropratio = 0.5

imgsize = 256 # 이미지는 256 사이즈

# Data Loader

training data

In [6]:
img_dir = "/content/drive/MyDrive/Colab Notebooks/심층학습_전이학습/clothes/train" #train data 내의 cats와 dogs 를 불러옴
train_data = dset.ImageFolder(img_dir, transforms.Compose([
                                                          transforms.CenterCrop(imgsize*2), 
                                                          transforms.RandomCrop(imgsize),
                                                          transforms.RandomHorizontalFlip(),
                                                          transforms.Resize(imgsize),
                                                          transforms.ToTensor()]))
print(train_data.__len__()) # train data가 몇 개 있는지

train_batch = data.DataLoader(train_data, batch_size = batch_size, shuffle = True, num_workers=2)

141


### 고정된 데이터 셋

In [8]:
# dev_data / val_data
img_dir = "/content/drive/MyDrive/Colab Notebooks/심층학습_전이학습/clothes/val"
dev_data = dset.ImageFolder(img_dir, transforms.Compose([
                                                         #transfroms.Scale(256),
                                                         #transfroms.RandomSizedCrop(224),

                                                         transforms.CenterCrop(size=imgsize),
                                                         transforms.Resize(imgsize),
                                                         transforms.ToTensor()]))

dev_batch = data.DataLoader(dev_data, batch_size= batch_size, shuffle=False, num_workers=2)

# 하이퍼파라미터 튜닝을 위한 것. 이로 가장 성능이 잘 나오는 하이퍼파라미터 셋을 설정해야함.
# 그 후 테스트 데이터로 성능 한 번 측정

In [9]:
# 3. Test data
img_dir = "/content/drive/MyDrive/Colab Notebooks/심층학습_전이학습/clothes/test"

test_data = dset.ImageFolder(img_dir, transforms.Compose([
                                                         #transfroms.Scale(256),
                                                         #transfroms.RandomSizedCrop(224),

                                                         transforms.CenterCrop(size=imgsize),
                                                         transforms.Resize(imgsize),
                                                         transforms.ToTensor()]))
test_batch = data.DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=2)

In [10]:
nclass = len(train_data.classes)
print(" # of classes : %d" %nclass)
print(train_data.classes)
print(train_data.class_to_idx)
print(train_data.__len__())

print("Training : %d, Dev : %d, Test : %d" %(train_data.__len__(),dev_data.__len__(),test_data.__len__() ))


 # of classes : 3
['hat', 'pants', 'tshirt']
{'hat': 0, 'pants': 1, 'tshirt': 2}
141
Training : 141, Dev : 56, Test : 70


In [11]:
print(train_data.classes)
print(dev_data.classes)
print(test_data.classes)

['hat', 'pants', 'tshirt']
['hat', 'pants', 'tshirt']
['hat', 'pants', 'tshirt']


# Model

1) Pretrained VGG Model

In [12]:
vgg = models.vgg19(pretrained=True)
# vgg19 라는 모델클래스 선언함. -> pretrained = True => 미리 학습된 파라미터 get
# 만약 False 면 구조만 가지고 오게 됨.
for name, module in vgg.named_children():
    print(name)

print(list(vgg.children())[0])
print(list(vgg.children())[-1])

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth



features
avgpool
classifier
Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (17): ReLU

In [13]:
print(list(vgg.children())[0][0])

Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


2) Customized Fully Model

In [14]:
base_dim = 64
fsize = imgsize/32

class MyVGG(nn.Module) : 
      def __init__(self):
        super(MyVGG, self).__init__()
        # [0] : features(conv), [1] : classifier(fc)
        self.layer0 = nn.Sequential(*list(vgg.children())[0])

        self.layer1 = nn.Sequential(  # 내가 대체하는 부분
            nn.Linear(int(8*base_dim * fsize * fsize), n_node),
            # n_node만 내가 원하는 값
            nn.BatchNorm1d(n_node),
            nn.ReLU(),
            nn.Dropout2d(dropratio),

            nn.BatchNorm1d(n_node),
            nn.ReLU(),
            nn.Dropout2d(dropratio),

            nn.Linear(n_node, n_node),
            nn.BatchNorm1d(n_node),
            nn.ReLU(),
            nn.Dropout2d(dropratio),

            nn.Linear(n_node, nclass), # 주의해야할 부분 -> 클래스가 몇 개인지
                )
        # weight initialization
        for m in self.layer1.modules() :
            if isinstance(m, nn.Conv2d) : 
                init.kaiming_normal(m.weight.data) # ReLU 일때
                m.bias.data.fill_(0)
            if isinstance(m, nn.Linear) : 
                init.kaiming_normal(m.weight.data) 
                m.bias.data.fill_(0)
        
        def forward(self, x) :
            # layer0의 사이즈를 무식하게 프린트하여 알아낼 숭 ㅣㅆ음
            # print(x.size())
            out = self.layer0(x)
            out = out.view(out.size(0), -1) # feature map을 한 줄로 펴달라는 의미
            out = self.layer1(out) # FC로 선언한 layer1에 넣어줘
            return out

3) Model on GPU

In [15]:
model = MyVGG().cuda()

for params in model.layer0.parameters():
    params.required_grad = False

for params in model.layer1.parameters():
    params.required_grad = True
    
for name in model.children():
      print(name)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.


Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (17): ReLU(inplace=True)
  (18): MaxPoo

# 4. Optimizer & Loss

In [16]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.layer1.parameters(), lr=learning_rate)

# 5. Train

In [17]:
import utils

total_time = 0
disp_step = 10

to_train = True

if (to_train == False):
    netname = "nets/catdog_vgg19_19.pkl"
    model = torch.load(netname)
else : 
    print("3 layer, n_node : %d, dropratio : %.2f" %(n_node, dropratio))
    model.eval()
    train_corr = utils.ComputeCorr(train_batch, model)
    dev_corr = utils.ComputeCorr(dev_batch, model)
    test_corr = utils.ComputeCorr(test_batch, model)
    print("Correct of train : %.2f, dev %.2f, test : %.2f" %(train_corr,dev_corr,test_corr))
    model.train()
    
    netname = 'nets/media_vgg19'

    # graph 그리기
    x_epoch = []
    y_train_err = []
    y_dev_err = []
    y_test_err = []
    
    x_epoch.append(0)
    y_train_err.append(100.0-train_corr)
    y_dev_err.append(100.0-dev_corr)
    y_test_err.append(100.0-test_corr)
    
#     # 학습을 재시작한다면
#     netname = '../nets/media_pre_vgg19.pkl'
#     model = torch.load(netname)
#     # 파라미터 학습 여부 결정
#     for params in model.layer0.parameters():
#         params.required_grad = False    
#     for params in model.layer1.parameters():
#         params.required_grad = True
#     for i in range(34, epoch):
    
    # 재시작하지 않는 다면
    for i in range(epoch):
        start_time = time.time()
        print("%d.." %i),
        for img,label in train_batch:
            img = Variable(img).cuda()
            label = Variable(label).cuda()

            optimizer.zero_grad()
            output = model(img)
            loss = loss_func(output,label)
            loss.backward()
            optimizer.step()

        end_time = time.time()
        duration = end_time - start_time
        total_time += duration
        if (i % disp_step == 0) or (i==epoch-1):
            torch.save(model, netname+'_%d.pkl'%i, )
            print("\n[%d/%d] loss: %.3f, " %(i, epoch, (loss.cpu()).data.numpy())),
            
            # train, dev, train accr
            model.eval()  # evaluation(test) mode 로 바꾸기 -> dropout, batch normalization 에 영향을 줌.
            train_corr = utils.ComputeCorr(train_batch, model)
            dev_corr = utils.ComputeCorr(dev_batch, model)
            test_corr = utils.ComputeCorr(test_batch, model)
            print("Correct of train: %.2f, dev: %.2f, test: %.2f, " 
                  %(train_corr, dev_corr, test_corr)),
            model.train()            
            print("time: %.2f sec.." %(total_time))
            
            # graph 그리기
            x_epoch.append(i+1)
            y_train_err.append(100.0-train_corr)
            y_dev_err.append(100.0-dev_corr)
            y_test_err.append(100.0-test_corr)
    print("Total time: %.2f sec" %total_time)

3 layer, n_node : 1024, dropratio : 0.50


NotImplementedError: ignored

In [ ]:
# epoch-err curve 
if (to_train):
    plt.plot(x_epoch, y_train_err, color='black', label='train err', linestyle='--')
    plt.plot(x_epoch, y_dev_err, color='red', label='dev err')
    plt.plot(x_epoch, y_test_err, color='blue', label='test err')
    
    plt.xlabel('epoch')
    plt.ylabel('err')
    plt.title('epoch & err graph')
    plt.legend(loc="upper right")
    plt.show()

# 6. Evaluation for dev & test data

In [ ]:
model.eval()   # evaluation(test) mode 로 바꾸기 -> dropout, batch normalization 에 영향을 줌.
utils.EvaluateClassifier(dev_batch, model, dev_data.classes, batch_size)

In [ ]:
model.eval()
_, _,_ = utils.EvaluateClassifier(test_batch, model, test_data.classes, batch_size)

In [ ]:
utils.VisTFPred(test_batch, model, test_data.classes, batch_size, i_n=2)